In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('news.csv')
df.dropna(inplace=True)
df.reset_index(inplace=True)
train, test = train_test_split(df, test_size=0.2)
train.reset_index(inplace=True)
test.reset_index(inplace=True)

In [2]:
import nltk
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')

In [3]:
import string
from num2words import num2words
from nltk.stem import WordNetLemmatizer
def stopword_fun(x):
    if len(x['content'].split('.')) == 1:
        text = x['content'].replace('  ','. ')
    else:
        text = x['content']
    words = [word for word in text.split() if word.lower() not in sw_nltk]
    text = " ".join(words)
    text = "".join([i for i in text if i not in '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~]'''])
    words = [WordNetLemmatizer().lemmatize(word) for word in text.split(' ')]
    text = ''
    for word in words:
        
        if word.isnumeric() :
            text  = text + " " +num2words(float(word))
        elif word[:-1].isnumeric():
            text  = text + " " +num2words(float(word[:-1])) + '.'
        elif len(word) >2:
            text =  text + " " + word
    words = [word for word in text.split() if word.lower() not in sw_nltk]    
    text = " ".join(words)
    text = "".join([i for i in text if i not in '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~]'''])
    text = text.lower()
    return text
def stopword_fun_title(x):
    if len(x['title'].split('.')) == 1:
        text = x['title'].replace('  ','. ')
    else:
        text = x['title']
    words = [word for word in text.split() if word.lower() not in sw_nltk]
    text = " ".join(words)
    text = "".join([i for i in text if i not in '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~]'''])
    words = [WordNetLemmatizer().lemmatize(word) for word in text.split(' ')]
    text = ''
    for word in words:
        
        if word.isnumeric() :
            text  = text + " " +num2words(float(word))
        elif word[:-1].isnumeric():
            text  = text + " " +num2words(float(word[:-1])) + '.'
        elif len(word) >2:
            text =  text + " " + word
    words = [word for word in text.split() if word.lower() not in sw_nltk]    
    text = " ".join(words)
    text = "".join([i for i in text if i not in '''!"#$%&'()*+,-/:;<=>?@[\]^_`{|}~]'''])
    text = text.lower()
    return text

content = train.apply(stopword_fun,axis=1)

title = train.apply(stopword_fun_title,axis=1)

In [5]:

def tokenize(content1):
    sent_dict = {}
    sentence = content1.split('. ')
    for j,i in enumerate(sentence):
        sent_dict[j] = i.split()
    return sent_dict
processed_text = {}


print(len(content),len(title))
for j,i in enumerate(content):

    processed_text[j] = tokenize(i)



644 644


In [6]:
DF = { i:{} for i in range(len(content))}
from collections import Counter
for i in range(len(content)):
  for j in processed_text[i].keys():
    tokens = processed_text[i][j]
    for w in tokens:
        if w in DF[i].keys():
            DF[i][w].add(j)
        else:
            DF[i][w] = {j}
   

for i in DF:
   for j in DF[i]:
     DF[i][j] = len(DF[i][j])

def doc_freq(word,j):
    c = 0
    if word in DF[j].keys():
        c = DF[j][word]
    else:
        pass
    return c


In [7]:
doc = 0


N = len(content)
tf_idf = { i:{} for i in range(len(content))}
for i in range(N):
  n = len (processed_text[i])
  tf_idf[doc] = { k:{} for k in range(n)}
  for j in processed_text[i].keys():
    
    tokens = processed_text[i][j]
    
    counter = Counter(tokens )
    words_count = len(tokens)
    
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df_val = doc_freq(token,i)
        idf = np.log((n+1)/(df_val+1))
        
        tf_idf[doc][j][token] = tf*idf

  doc += 1


In [8]:
threshold = 0
average_tfidf_sent = []
max = 0
minimum = 0
for i in tf_idf:
    ad = []
    
    for j in tf_idf[i]:
        n = len(tf_idf[i][j])
        average_tfidf = sum(tf_idf[i][j].values())/n
        if average_tfidf > max:
            max = average_tfidf
        if average_tfidf < minimum:
            minimum = average_tfidf
        ad.append(average_tfidf)
    average_tfidf_sent.append(ad)
print(max,minimum)  
threshold = minimum +  (max - minimum)/19
threshold

3.4011973816621555 0


0.17901038850853448

In [18]:
def doc_freq2(word):
    c = 0
    if word in DF.keys():
        c = DF[word]
    else:
        pass
    return c
def remove_sentence(x,threshold):
    text = x['content_mod']
    processed_text = tokenize(text)
    DF={}
    for j in processed_text.keys():
        tokens = processed_text[j]
        for w in tokens:
            if w in DF.keys():
                DF[w].add(j)
            else:
                DF[w] = {j}
       
    for i in DF:
         DF[i] = len(DF[i])
    doc = 0
    n = len(processed_text)
    tf_idf = {i:{} for i in range(n)}
    for i in range(n):
        
        tokens = processed_text[i]
        
        counter = Counter(tokens )
        words_count = len(tokens)
        
        for token in np.unique(tokens):
            
            tf = counter[token]/words_count
            df_val = doc_freq2(token)
            idf = np.log((n+1)/(df_val+1))
            
            tf_idf[i][token] = tf*idf
    
        doc += 1
    remove = []
    retain = []
    for i in tf_idf:
        n = len(tf_idf[i])
        if sum(tf_idf[i].values())/n < threshold:
          
            remove.append(i)
        else:
           
            retain.append(i)
    remove_text = ''
    retain_text = ''
    for i in remove:
        remove_text +=  ' '.join(processed_text[i])+ '. ' 
    for i in retain:
        retain_text +=  ' '.join(processed_text[i]) + '. ' 
    x['new_content'] = retain_text
    x['removed_lines'] = remove_text
    return x

test['content_mod'] = test.apply(stopword_fun,axis=1)
test = test.apply(remove_sentence,args=[threshold],axis=1)
test[['content','removed_lines','new_content']].to_csv('result.csv')

            
            
            